In [3]:
import numpy as np
import pylab as pylab

In [9]:
A1=np.array([[1,3,5,1],[2,4,5,6],[1,3,5,7]])
u, s, v = pylab.linalg.svd(A1)
u

array([[-0.3743954 ,  0.9007223 ,  0.22028941],
       [-0.65341629, -0.08770773, -0.7519006 ],
       [-0.65793255, -0.42544882,  0.62138399]])

In [55]:
A1.sum(axis=1)

array([10, 17, 16])

In [56]:
def norm_block(B):
    """
    Average row summ (1-norm).
    """
    
    s=B.sum(axis=1)
    ss=s.sum()
    return ss/len(B)

In [103]:
a=np.array([[1,2],[3,4]])
a.shape == (2,2)

True

In [110]:
a=np.array([[1,2,4],[4,5,6]])
b=np.array([[1,2,6],[4,5,6]])
all([all(a[0,:]==b[0,:]),all(a[1,:]==b[1,:])])

False

In [151]:
def block_in(block,l):
    """
    Checks whether a block is in a list of blocks
    
    """

    sb=block.shape
    for i in range(len(l)):
        if (l[i]).shape == sb:
            L=[]
            for j in range(len(l[i])):
                L.append(all(l[i][j,:]==block[j,:]))
            if all(L):
                return True
    return False
            

In [152]:
def find_meta_states(B,threshold=0.95,list_of_blocks=[]):
    """
    Singular Value Decomposition based method for finding the metastable states.
    
    INPUT:
        - B = A markov transition matrix
        - threshold = 
        
    OUTPUT:
        Number m and sizes n_i, i = 1,...,m of identified blocks in B, and a permutation
        matrix P such that P * B * np.transpose(P) = A + E
    """
    
    d1 = len(B)
    d2 = len(B[0])
    
    if d1==d2:
        d=d1
    else:
        return "Input error"
    
    u, s, v = pylab.linalg.svd(B)
    u2 = u[:,1]
    u2_indsort = np.argsort(u2)
    
    #get the permutation matrix P
    P = np.zeros([d,d])
    for i in range(d):
        P[i,u2_indsort[i]] = 1
        
    #apply the permutation to B
    B_ = np.dot(np.dot(P,B),np.transpose(P))
    
    #identify two possible blocks B1, B2 by looking at the sign change in u2
    if u2[0] > 0:
        k=0
        while u2[k] > 0:
            k=k+1
    elif u2[0] < 0:
        k=0
        while u2[k] < 0:
            k = k+1
    else:
        k=0
        while u2[k] == 0:
            k = k+1
    
    k_ = [i for i in range(k)]
    _k = [i for i in range(d) if i not in k_]
    
    #two blocks
    B1=B[k_,:]
    B1=B1[:,k_]
    
    B2=B[_k,:]
    B2=B2[:,_k]
    
    if norm_block(B1) > threshold and norm_block(B2) > threshold:
        #we have identified two blocks and separate them
        list_of_blocks.append(B1)
        list_of_blocks.append(B2)
        l1 = find_meta_states(B1,threshold,list_of_blocks)
        list_of_blocks = find_meta_states(B2,threshold,l1)
        
    else:
        #THE CURRENT BLOCK CANNOT BE FURTHER REDUCED
        if not(block_in(B,list_of_blocks)):
            list_of_blocks.append(B)
        return list_of_blocks
    return list_of_blocks
        
    
        

In [153]:
A_ex = np.array([[0.2,0.8,0,0,0],[0.4,0.6,0,0,0],[0,0,0.3,0.3,0.4],[0,0,0.2,0.2,0.6],[0,0,0.1,0.1,0.8]])
A_ex

array([[ 0.2,  0.8,  0. ,  0. ,  0. ],
       [ 0.4,  0.6,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.3,  0.3,  0.4],
       [ 0. ,  0. ,  0.2,  0.2,  0.6],
       [ 0. ,  0. ,  0.1,  0.1,  0.8]])

In [154]:
find_meta_states(A_ex)

[array([[ 0.2,  0.8],
        [ 0.4,  0.6]]), array([[ 0.3,  0.3,  0.4],
        [ 0.2,  0.2,  0.6],
        [ 0.1,  0.1,  0.8]])]

In [155]:
find_meta_states(T,0.95,[])

[array([[ 0.9 ,  0.1 ],
        [ 0.1 ,  0.89]]), array([[ 0.79,  0.2 ],
        [ 0.2 ,  0.8 ]])]

In [158]:
A2=np.random.rand(7,7)
print(A2)
find_meta_states(A2,0.95,[])

[[ 0.65984865  0.08709168  0.85317769  0.68616699  0.11446422  0.30125339
   0.19676675]
 [ 0.97801328  0.80919581  0.05966196  0.63991296  0.58701481  0.29925951
   0.4318922 ]
 [ 0.68621717  0.22422049  0.0297601   0.54816     0.9591874   0.29421971
   0.59955607]
 [ 0.25670608  0.31917239  0.76908987  0.36143229  0.58625213  0.94272902
   0.73199731]
 [ 0.97115466  0.9656135   0.42599721  0.36848566  0.04026076  0.963847
   0.92512352]
 [ 0.58372315  0.6891781   0.2475603   0.93265183  0.78590838  0.01468832
   0.61219702]
 [ 0.85748253  0.71197175  0.03585486  0.78202315  0.56173212  0.75463372
   0.98163075]]


[array([[ 0.65984865,  0.08709168,  0.85317769,  0.68616699,  0.11446422,
          0.30125339,  0.19676675],
        [ 0.97801328,  0.80919581,  0.05966196,  0.63991296,  0.58701481,
          0.29925951,  0.4318922 ],
        [ 0.68621717,  0.22422049,  0.0297601 ,  0.54816   ,  0.9591874 ,
          0.29421971,  0.59955607],
        [ 0.25670608,  0.31917239,  0.76908987,  0.36143229,  0.58625213,
          0.94272902,  0.73199731],
        [ 0.97115466,  0.9656135 ,  0.42599721,  0.36848566,  0.04026076,
          0.963847  ,  0.92512352],
        [ 0.58372315,  0.6891781 ,  0.2475603 ,  0.93265183,  0.78590838,
          0.01468832,  0.61219702],
        [ 0.85748253,  0.71197175,  0.03585486,  0.78202315,  0.56173212,
          0.75463372,  0.98163075]])]

In [35]:
T=np.array([[0.9,0.1,0,0],[0.1,0.89,0.01,0],[0,0.01,0.79,0.2],[0,0,0.2,0.8]])
u, s, v = pylab.linalg.svd(T)
u2 = u[:,1]
print(u2)
print(np.sort(u2))
print(np.argsort(u2))
print(np.sign(u2))

[ 0.52486131  0.4743488  -0.48728825 -0.51192186]
[-0.51192186 -0.48728825  0.4743488   0.52486131]
[3 2 1 0]
[ 1.  1. -1. -1.]


In [48]:
T=np.array([[0.9,0.1,0,0],[0.1,0.89,0.01,0],[0,0.01,0.79,0.2],[0,0,0.2,0.8]])
A=np.copy(T)
print(A)
n=len(T)
k=2
k_ = [i for i in range(k)]
_k = [i for i in range(n) if i not in k_]
print(k_)
print(_k)

[[ 0.9   0.1   0.    0.  ]
 [ 0.1   0.89  0.01  0.  ]
 [ 0.    0.01  0.79  0.2 ]
 [ 0.    0.    0.2   0.8 ]]
[0, 1]
[2, 3]


In [53]:
T1= A[k_,:]
T1= T1[:,k_]
T1

[[ 0.9   0.1   0.    0.  ]
 [ 0.1   0.89  0.01  0.  ]
 [ 0.    0.01  0.79  0.2 ]
 [ 0.    0.    0.2   0.8 ]]


array([[ 0.9 ,  0.1 ],
       [ 0.1 ,  0.89]])

In [50]:
np.sort?

In [31]:
np.ndarray.sort?

In [32]:
np.argsort?